In [1]:
from scipy.io import wavfile
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
%matplotlib inline

In [2]:
in_memory_wav = wavfile.read("morse.wav")

In [3]:
small_df = pd.DataFrame(in_memory_wav[1][:,1], columns=["left"])
small_df["sample_loc"] = small_df.index / 44100
tiny_df = small_df[small_df.left > 10000]

time_stamps = [item[1].sample_loc for item in tiny_df.iterrows()]

In [4]:
def get_letter_list(time_stamps):
    old_loc = time_stamps[0]
    stop_locs = [0]
    letter_list = []

    for idx, ts in enumerate(time_stamps):
        if idx != 0:
            val = (time_stamps[idx - 1] - old_loc) * 10
        else:
            val = 0

        old_loc = time_stamps[idx - 1]
        if val > .4:
            stop_locs.append(idx)
            if val > 10:
                letter_list.append(stop_locs)
                stop_locs = [idx]
                continue

    stop_locs.append(len(time_stamps))
    letter_list.append(stop_locs)
    return letter_list


In [5]:
def clean_gaps(letter_list):
    gaps = []

    for letter in letter_list:
        current_letter = []
        for idx, loc in enumerate(letter):
            if idx > 0:
                current_letter.append(loc - letter[idx - 1])
        gaps.append(current_letter)
    return gaps

In [6]:
def get_cw(gaps):
    return [list(map(lambda x: "." if x < 4500 else "-", letter)) for letter in gaps]

In [7]:
def clean_cw(cw):
    return list(map(lambda x: "".join(x), cw))

In [8]:
from string import ascii_letters

def interpret_cw(cleaned_cw):
    morse_dict = dict(zip(map(lambda x: "".join(x), cw), ascii_letters))
    return "".join(list(map(lambda _: morse_dict.get(_), cleaned_cw)))

In [9]:
letter_list = get_letter_list(time_stamps)
cleaned_gaps = clean_gaps(letter_list)
cw = get_cw(cleaned_gaps)
cleaned_cw = clean_cw(cw)
plain_english = interpret_cw(cleaned_cw)

In [10]:
print(plain_english)

abcdefghijklmnopqrstuvwxyz
